# Convolutional Neural Networks

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.auto import tqdm

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters
NUM_EPOCHS = 5
NUM_CLASSES = 10
BATCH_SIZE = 100
LEARNING_RATE = 0.001

In [4]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root='../data',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='../data',
    train=False,
    transform=transforms.ToTensor()
)


# Data loader
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [5]:
class ConvNet(nn.Module):
    """
    Convolutional Neural Network (Two Convolutional Layers)
    """

    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = nn.Linear(in_features=7*7*32, out_features=num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1) # Flatten
        out = self.fc(out)
        return out

In [6]:
model = ConvNet(num_classes=NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [7]:
# Train the model
total_step = len(train_loader)
for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad() # Clear gradients
        loss.backward() # Backpropagation
        optimizer.step() # Update weights

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [100/600], Loss: 0.2280
Epoch [1/5], Step [200/600], Loss: 0.1393
Epoch [1/5], Step [300/600], Loss: 0.1386
Epoch [1/5], Step [400/600], Loss: 0.0409
Epoch [1/5], Step [500/600], Loss: 0.0436
Epoch [1/5], Step [600/600], Loss: 0.0956
Epoch [2/5], Step [100/600], Loss: 0.0226
Epoch [2/5], Step [200/600], Loss: 0.0323
Epoch [2/5], Step [300/600], Loss: 0.0183
Epoch [2/5], Step [400/600], Loss: 0.0273
Epoch [2/5], Step [500/600], Loss: 0.0061
Epoch [2/5], Step [600/600], Loss: 0.1575
Epoch [3/5], Step [100/600], Loss: 0.0124
Epoch [3/5], Step [200/600], Loss: 0.0426
Epoch [3/5], Step [300/600], Loss: 0.0678
Epoch [3/5], Step [400/600], Loss: 0.0020
Epoch [3/5], Step [500/600], Loss: 0.0139
Epoch [3/5], Step [600/600], Loss: 0.0294
Epoch [4/5], Step [100/600], Loss: 0.0105
Epoch [4/5], Step [200/600], Loss: 0.0149
Epoch [4/5], Step [300/600], Loss: 0.0125
Epoch [4/5], Step [400/600], Loss: 0.0024
Epoch [4/5], Step [500/600], Loss: 0.0223
Epoch [4/5], Step [600/600], Loss:

In [8]:
# Test the model

## Set the model to evaluation mode
## -- BatchNorm uses moving mean/variance instead of mini-batch mean/variance
model.eval() 

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.97 %


In [9]:
# Save the model checkpoint
torch.save(model.state_dict(), 'cnn-model.ckpt')